In [2]:
pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 12.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [ ]:
train_data=pd.read_csv("trainschybou.csv")
test_data=pd.read_csv("test_data.csv")
train_data=train_data.drop("I am currently employed at least part-time",axis=1)
test_data=test_data.drop("I am currently employed at least part-time",axis=1)
columns_order = train_data.columns.tolist()
test_data = test_data[columns_order]
y_train = train_data['Unemployed']
X_train = train_data.drop(columns=('Unemployed'))
y_test = test_data['Unemployed']
X_test = test_data.drop(columns=('Unemployed'))

# Pre výber hyperparametrov bol použity nastroj Optuna.  Každý kód pre každý model som použila vela krát s rôznym počtom n_trials a najlepšie parametre už boli použité pri modelovaní

# KNN

In [ ]:
import optuna
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
import pandas as pd

def objective(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 1, 50)
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
    p = trial.suggest_int('p', 1, 5) 
    leaf_size = trial.suggest_int('leaf_size', 10, 50)
    algorithm = trial.suggest_categorical('algorithm', ['auto', 'ball_tree', 'kd_tree', 'brute'])
    model = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights, p=p, leaf_size=leaf_size, algorithm=algorithm)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    f1_class1 = f1_score(y_test, y_pred, average='binary', pos_label=1)
    return f1_class1
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=150)
best_params = study.best_params
print("Best Parameters:", best_params)
best_model = KNeighborsClassifier(**best_params)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
f1_class1 = f1_score(y_test, y_pred, average='binary', pos_label=1)
print("F1 Score for class 1:", f1_class1)
print("Classification Report:")
print(classification_report(y_test, y_pred))
y_train_first_model = train_data['Unemployed']
X_train_first_model = train_data.drop(columns=('Unemployed'))
y_test_first_model = test_data['Unemployed']
X_test_first_model = test_data.drop(columns=('Unemployed'))

In [ ]:
from imblearn.over_sampling import ADASYN
def objective(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 1, 40)
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
    p = trial.suggest_int('p', 1, 5)  
    leaf_size = trial.suggest_int('leaf_size', 10, 100)
    algorithm = trial.suggest_categorical('algorithm', ['auto', 'ball_tree', 'kd_tree', 'brute'])
    n_neighbors_adasyn = trial.suggest_int('n_neighbors_adasyn', 1, 50)  
    model = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights, p=p, leaf_size=leaf_size, algorithm=algorithm)
    adasyn = ADASYN(n_neighbors=n_neighbors_adasyn, random_state=42)
    X_train_resampled, y_train_resampled = adasyn.fit_resample(X_train, y_train)
    
    model.fit(X_train_resampled, y_train_resampled)
    
    y_pred = model.predict(X_test)

    f1_class1 = f1_score(y_test, y_pred, average='binary', pos_label=1)

    return f1_class1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

best_params = study.best_params
print("Best Parameters:", best_params)

best_model = KNeighborsClassifier(**{k: v for k, v in best_params.items() if k != 'n_neighbors_adasyn'})

adasyn = ADASYN(n_neighbors=best_params['n_neighbors_adasyn'], random_state=42)
X_train_resampled, y_train_resampled = adasyn.fit_resample(X_train, y_train)
best_model.fit(X_train_resampled, y_train_resampled)

y_pred = best_model.predict(X_test)

f1_class1 = f1_score(y_test, y_pred, average='binary', pos_label=1)
print("F1 Score for class 1:", f1_class1)

print("Classification Report:")
print(classification_report(y_test, y_pred))


In [ ]:
from imblearn.over_sampling import SMOTE
def objective(trial):

    n_neighbors = trial.suggest_int('n_neighbors', 1, 50)
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
    p = trial.suggest_int('p', 1, 5)  
    leaf_size = trial.suggest_int('leaf_size', 10, 60)
    algorithm = trial.suggest_categorical('algorithm', ['auto', 'ball_tree', 'kd_tree', 'brute'])
    k_neighbors_smote = trial.suggest_int('k_neighbors_smote', 1, 35)  # Число соседей для SMOTE
    model = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights, p=p, leaf_size=leaf_size, algorithm=algorithm)

    smote = SMOTE(k_neighbors=k_neighbors_smote, random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    
    model.fit(X_train_resampled, y_train_resampled)
    
    y_pred = model.predict(X_test)
    
    f1_class1 = f1_score(y_test, y_pred, average='binary', pos_label=1)
    
    return f1_class1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=250)

best_params = study.best_params
print("Best Parameters:", best_params)

best_model = KNeighborsClassifier(**{k: v for k, v in best_params.items() if k != 'k_neighbors_smote'})

smote = SMOTE(k_neighbors=best_params['k_neighbors_smote'], random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
best_model.fit(X_train_resampled, y_train_resampled)

y_pred = best_model.predict(X_test)

f1_class1 = f1_score(y_test, y_pred, average='binary', pos_label=1)
print("F1 Score for class 1:", f1_class1)

print("Classification Report:")
print(classification_report(y_test, y_pred))


# DT

In [ ]:
from sklearn.tree import DecisionTreeClassifier

def objective(trial):
    max_depth = trial.suggest_int('max_depth', 3, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 50)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 50)
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    
    model = DecisionTreeClassifier(
        criterion=criterion,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )
    
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    f1_class1 = f1_score(y_test, y_pred, average='binary', pos_label=1)
    
    return f1_class1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

best_params = study.best_params
print("Best Parameters:", best_params)

best_model = DecisionTreeClassifier(**best_params, random_state=42)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)

f1_class1 = f1_score(y_test, y_pred, average='binary', pos_label=1)
print("F1 Score for class 1:", f1_class1)

print("Classification Report:")
print(classification_report(y_test, y_pred))


In [ ]:
def objective(trial):
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    max_depth = trial.suggest_int('max_depth', 1, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 30)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 30)
    k_neighbors = trial.suggest_int('k_neighbors', 5, 40)

    smote = SMOTE(random_state=42, k_neighbors=k_neighbors, sampling_strategy='minority')
    X_resampled_train, y_resampled_train = smote.fit_resample(X_train, y_train)
    
    model = DecisionTreeClassifier(criterion=criterion,
                                   max_depth=max_depth,
                                   min_samples_split=min_samples_split,
                                   min_samples_leaf=min_samples_leaf,
                                   random_state=42)
    
    model.fit(X_resampled_train, y_resampled_train)
    
    y_pred = model.predict(X_test)
    
    f1_class_1 = f1_score(y_test, y_pred, pos_label=1)
    
    return f1_class_1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=250)

best_params = study.best_params
k_neighbors = best_params.pop('k_neighbors')

print("Best Parameters:")
print("  Criterion:", best_params['criterion'])
print("  Max Depth:", best_params['max_depth'])
print("  Min Samples Split:", best_params['min_samples_split'])
print("  Min Samples Leaf:", best_params['min_samples_leaf'])
print("  Number of Neighbors (for SMOTE):", k_neighbors)

smote = SMOTE(random_state=42, k_neighbors=k_neighbors, sampling_strategy='minority')
X_resampled_train, y_resampled_train = smote.fit_resample(X_train, y_train)

best_model = DecisionTreeClassifier(**best_params, random_state=42)
best_model.fit(X_resampled_train, y_resampled_train)

y_pred_best = best_model.predict(X_test)

f1_class_1_best = f1_score(y_test, y_pred_best, pos_label=1)
print("F1 Score for class 1 with best parameters:", f1_class_1_best)

print("Classification Report:")
print(classification_report(y_test, y_pred_best))


In [ ]:
def objective(trial):
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    max_depth = trial.suggest_int('max_depth', 1, 100)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 30)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 30)
    n_neighbors = trial.suggest_int('n_neighbors', 5, 50)

    adasyn = ADASYN(random_state=42, n_neighbors=n_neighbors, sampling_strategy='minority')
    X_resampled_train, y_resampled_train = adasyn.fit_resample(X_train, y_train)
    
    model = DecisionTreeClassifier(criterion=criterion,
                                   max_depth=max_depth,
                                   min_samples_split=min_samples_split,
                                   min_samples_leaf=min_samples_leaf,
                                   random_state=42)
    
    model.fit(X_resampled_train, y_resampled_train)
    
    y_pred = model.predict(X_test)
    
    f1_class_1 = f1_score(y_test, y_pred, pos_label=1)
    
    return f1_class_1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=250)

best_params = study.best_params
n_neighbors = best_params.pop('n_neighbors')

print("Best Parameters:")
print("  Criterion:", best_params['criterion'])
print("  Max Depth:", best_params['max_depth'])
print("  Min Samples Split:", best_params['min_samples_split'])
print("  Min Samples Leaf:", best_params['min_samples_leaf'])
print("  Number of Neighbors (for ADASYN):", n_neighbors)

adasyn = ADASYN(random_state=42, n_neighbors=n_neighbors, sampling_strategy='minority')
X_resampled_train, y_resampled_train = adasyn.fit_resample(X_train, y_train)

best_model = DecisionTreeClassifier(**best_params, random_state=42)
best_model.fit(X_resampled_train, y_resampled_train)

y_pred_best = best_model.predict(X_test)

f1_class_1_best = f1_score(y_test, y_pred_best, pos_label=1)
print("F1 Score for class 1 with best parameters:", f1_class_1_best)

print("Classification Report:")
print(classification_report(y_test, y_pred_best))

# RF

In [ ]:
from sklearn.ensemble import RandomForestClassifier
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 100)
    max_depth = trial.suggest_int('max_depth', 5, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)
    
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        criterion="entropy",
        random_state=42
    )
    
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    f1_class1 = f1_score(y_test, y_pred, pos_label=1)
    
    return f1_class1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=250)

best_params = study.best_params
print("Best Parameters:", best_params)

best_model = RandomForestClassifier(**best_params, criterion="gini", random_state=42)

best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)

f1_class1 = f1_score(y_test, y_pred, pos_label=1)
print("F1 Score for class 1:", f1_class1)

print("Classification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    max_depth = trial.suggest_int('max_depth', 5, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 40)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 30)
    k_neighbors = trial.suggest_int('k_neighbors', 5, 50)
    
    smote = SMOTE(random_state=42, k_neighbors=k_neighbors, sampling_strategy='minority')
    X_resampled_train, y_resampled_train = smote.fit_resample(X_train, y_train)
    
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        criterion="entropy",
        random_state=42
    )
    
    model.fit(X_resampled_train, y_resampled_train)
    
    y_pred = model.predict(X_test)
    
    f1_class1 = f1_score(y_test, y_pred, pos_label=1)
    
    return f1_class1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

best_params = study.best_params
print("Best Parameters:", best_params)

smote = SMOTE(random_state=42, k_neighbors=best_params['k_neighbors'], sampling_strategy='minority')
X_resampled_train, y_resampled_train = smote.fit_resample(X_train, y_train)

best_model = RandomForestClassifier(n_estimators=best_params['n_estimators'],
                                    max_depth=best_params['max_depth'],
                                    min_samples_split=best_params['min_samples_split'],
                                    min_samples_leaf=best_params['min_samples_leaf'],
                                    criterion="entropy",
                                    random_state=42)

best_model.fit(X_resampled_train, y_resampled_train)

y_pred = best_model.predict(X_test)

f1_class1 = f1_score(y_test, y_pred, pos_label=1)
print("F1 Score for class 1:", f1_class1)

print("Classification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 50)
    max_depth = trial.suggest_int('max_depth', 5, 100)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 30)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 30)
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    n_neighbors = trial.suggest_int('n_neighbors', 5, 50)
    
    adasyn = ADASYN(random_state=42, n_neighbors=n_neighbors, sampling_strategy='minority')
    X_resampled_train, y_resampled_train = adasyn.fit_resample(X_train, y_train)
    
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        criterion=criterion,
        random_state=42
    )
    
    model.fit(X_resampled_train, y_resampled_train)
    
    y_pred = model.predict(X_test)
    
    f1_class1 = f1_score(y_test, y_pred, pos_label=1)
    
    return f1_class1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

best_params = study.best_params
print("Best Parameters:", best_params)

adasyn = ADASYN(random_state=42, n_neighbors=best_params['n_neighbors'], sampling_strategy='minority')
X_resampled_train, y_resampled_train = adasyn.fit_resample(X_train, y_train)

best_model = RandomForestClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    criterion=best_params['criterion'],
    random_state=42
)

best_model.fit(X_resampled_train, y_resampled_train)

y_pred = best_model.predict(X_test)

f1_class1 = f1_score(y_test, y_pred, pos_label=1)
print("F1 Score for class 1:", f1_class1)

print("Classification Report:")
print(classification_report(y_test, y_pred))


# Xgboost

In [ ]:
import xgboost as xgb
from imblearn.pipeline import Pipeline

def objective(trial):
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'use_label_encoder': False,
        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
        'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'eta': trial.suggest_loguniform('eta', 1e-8, 1.0),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0, step=0.1),
        'tree_method': 'hist'
    }
    
    model = xgb.XGBClassifier(**params)
    
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    f1_class1 = f1_score(y_test, y_pred, pos_label=1)
    
    return f1_class1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

best_params = study.best_params
print("Best Parameters:", best_params)

best_model = xgb.XGBClassifier(**best_params)

best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))


In [ ]:
def objective(trial):
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'use_label_encoder': False,
        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
        'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'eta': trial.suggest_loguniform('eta', 1e-8, 1.0),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0, step=0.1),
        'tree_method': 'hist'
    }
    
    model = xgb.XGBClassifier(**params)
    
    smote = SMOTE(random_state=42, k_neighbors=trial.suggest_int('k_neighbors', 1, 50))
    
    pipeline = Pipeline([
        ('smote', smote),
        ('model', model)
    ])
    
    pipeline.fit(X_train, y_train)
    
    y_pred = pipeline.predict(X_test)
    
    f1_class1 = f1_score(y_test, y_pred, pos_label=1)
    
    return f1_class1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

best_params = study.best_params
print("Best Parameters:", best_params)

best_smote = SMOTE(random_state=42, k_neighbors=best_params['k_neighbors'])

best_model = xgb.XGBClassifier(**best_params)

best_pipeline = Pipeline([
    ('smote', best_smote),
    ('model', best_model)
])

best_pipeline.fit(X_train, y_train)

y_pred = best_pipeline.predict(X_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
def objective(trial):
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'use_label_encoder': False,
        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
        'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'eta': trial.suggest_loguniform('eta', 1e-8, 1.0),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0, step=0.1),
        'tree_method': 'hist'
    }
    
    model = xgb.XGBClassifier(**params)
    
    adasyn = ADASYN(random_state=42, n_neighbors=trial.suggest_int('n_neighbors', 1, 50))
    
    pipeline = Pipeline([
        ('adasyn', adasyn),
        ('model', model)
    ])
    
    pipeline.fit(X_train, y_train)
    
    y_pred = pipeline.predict(X_test)
    
    f1_class1 = f1_score(y_test, y_pred, pos_label=1)
    
    return f1_class1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

best_params = study.best_params
print("Best Parameters:", best_params)

best_adasyn = ADASYN(random_state=42, n_neighbors=best_params['n_neighbors'])

best_model = xgb.XGBClassifier(**best_params)

best_pipeline = Pipeline([
    ('adasyn', best_adasyn),
    ('model', best_model)
])

best_pipeline.fit(X_train, y_train)

y_pred = best_pipeline.predict(X_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))